In [11]:
pip install beautifulsoup4 requests pandas

Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import library
import bs4
import pandas as pd
import requests

In [13]:
url = 'https://fcstats.com/table,premier-league-england,1,1.php'

In [14]:
def get_page_contents(url):
    page = requests.get(url, headers={"Accept-Language": "en-US"})
    return bs4.BeautifulSoup(page.text, "html.parser")

In [15]:
soup = get_page_contents(url)
# print(soup.prettify())

In [16]:
tableResult = soup.findAll('table', class_='table evenOdd sortable tips')
# tableResult

In [17]:
content = soup.findAll('tr', class_='greenHover')[0]
# resultado = [result.find('a') for result in content]
len(content)
content

<tr class="greenHover" id="team_11">
<td style="background-color: #96b900 !important; color: #fff;" title="UEFA Champions League">1</td>
<td class="teamName"><a href="club,statistics,liverpool,11.php">Liverpool</a></td> <td class="leftBorder">22</td>
<td>21</td>
<td>1</td>
<td>0</td>
<td>52</td>
<td>14</td>
<td class="leftBorder rightBorder darkBackground">64</td>
<td class="leftBorder leftText sequencesMatches">
<a href="match,leicester-city,liverpool,389409.php" id="match_389409_opponent_30" title="Leicester City - Liverpool 0:4">
<span class="boxIcon boxIconW"></span> </a>
<a href="match,liverpool,wolverhampton-wanderers,389417.php" id="match_389417_opponent_20" title="Liverpool - Wolverhampton Wanderers 1:0">
<span class="boxIcon boxIconW"></span> </a>
<a href="match,liverpool,sheffield-united,389427.php" id="match_389427_opponent_40" title="Liverpool - Sheffield United 2:0">
<span class="boxIcon boxIconW"></span> </a>
<a href="match,tottenham-hotspur,liverpool,389442.php" id="matc

In [18]:
# sample get name team
team_name = content.findAll('a')[0].text
print('team name: ' + team_name)

team name: Liverpool


In [19]:
# sample get game and result
print('game and result: ' + content.findAll('a')[1].get('title'))  

game and result: Leicester City - Liverpool 0:4


In [20]:
#get number goal team todo game
find_team = content.findAll('a')[1].get('title').split('-')

if (find_team[0].strip() == team_name):
    number_space = content.findAll('a')[1].get('title').split(' ')
    goal_todo_team = number_space[len(number_space)-1].split(':')[0]
else:
    goal_todo_team = content.findAll('a')[1].get('title').split(':')[1]

goal_todo_team

'4'

In [21]:
# sample get id team
soup.findAll('tr', class_='greenHover')[0].get('id')

'team_11'

In [22]:
# ids_team = soup.findAll('tr', class_='greenHover')
# print(ids_team[0].get('id'))

In [23]:
#example insert data in dataframe
import pandas as pd

cars = {'Brand': ['Honda Civic','Toyota Corolla','Ford Focus','Audi A4'],
        'Price': [22000,25000,27000,35000]
        }

df = pd.DataFrame(cars, columns = ['Brand', 'Price'])

print (df)

            Brand  Price
0     Honda Civic  22000
1  Toyota Corolla  25000
2      Ford Focus  27000
3         Audi A4  35000


In [24]:
def getGoalTeamTodo(dataTeam):
    team_name = dataTeam.findAll('a')[0].text
    
    str_goalTeamTodoOfGame = ''
    
    for x in range(1,6):
        find_team = dataTeam.findAll('a')[x].get('title').split('-')

        if (find_team[0].strip() == team_name):
            number_space = dataTeam.findAll('a')[x].get('title').split(' ')
            goal_todo_team = number_space[len(number_space)-1].split(':')[0]
        else:
            goal_todo_team = dataTeam.findAll('a')[x].get('title').split(':')[1]

        if (str_goalTeamTodoOfGame == ''):
            str_goalTeamTodoOfGame = goal_todo_team
        else:
            str_goalTeamTodoOfGame = str_goalTeamTodoOfGame + '|' + goal_todo_team
    
    return str_goalTeamTodoOfGame

In [25]:
def getNameTeam(team):
    return team.findAll('a')[0].text

In [26]:
# create dataframe table teams and goals of games

# columns
columns = ['id_team', 'name_team', 'game1', 'game2', 'game3', 'game4', 'game5', 'id_team_next_game']

df_table = pd.DataFrame(columns=columns)

In [27]:
# get id of team next game
def getIdTeamNextGame(team):
    last_item_team = team.findAll('a')[len(team.findAll('a'))-1].get('id').split('_')
    id_team = last_item_team[len(last_item_team)-1] 
    print(id_team)
    #.split('-')
    return 'team_' + id_team

In [28]:
# insert data team in dataframe of table

def insert(df, row):
    insert_loc = df.index.max()

    if pd.isna(insert_loc):
        df.loc[0] = row
    else:
        df.loc[insert_loc + 1] = row

In [29]:
def getDataTeams(soup):
    for team in soup.findAll(attrs= {"class":"greenHover"}):
        #print('id_team: ' + team.get('id') + ' name team: ' + getNameTeam(team) + ' goal team todo: ' + getGoalTeamTodo(team))
        resultFiveLastGame = getGoalTeamTodo(team)
        result_games = resultFiveLastGame.split('|')
        # print(len(result_games))
        #df_table.append({
        #    'id_team':team.get('id'), 
        #    'name_team':getNameTeam(team), 
        #    'game1':result_games[0], 
        #    'game2':result_games[1], 
        #    'game3':result_games[2],
        #    'game4':result_games[3],
        #    'game5':result_games[4],
        #    'id_team_next_game':[]}, ignore_index = True) 
        insert(df_table, [team.get('id'), getNameTeam(team), result_games[0], result_games[1], result_games[2], result_games[3], result_games[4], getIdTeamNextGame(team)])
        

        
        

In [30]:
getDataTeams(soup)


20
40
18
1
6
11
12
59
61
7
32
8
26
13
72
30
2
43
46
17


In [31]:
df_table

,id_team,name_team,game1,game2,game3,game4,game5,id_team_next_game
0,team_11,Liverpool,4,1,2,1,2,team_20
1,team_12,Manchester City,2,2,2,6,2,team_40
2,team_30,Leicester City,0,2,3,1,1,team_18
3,team_7,Chelsea London,0,2,1,3,0,team_1
4,team_13,Manchester United,4,2,0,4,0,team_6
5,team_20,Wolverhampton Wanderers,3,0,1,1,3,team_11
6,team_40,Sheffield United,1,0,0,1,1,team_12
7,team_17,Tottenham Hotspur,2,2,0,0,0,team_59
8,team_26,Crystal Palace,2,1,1,1,2,team_61
9,team_1,Arsenal London,1,1,2,1,1,team_7


In [32]:
for team in soup.findAll(attrs= {"class":"greenHover"}):
    print(len(team.findAll('a')))

8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
